In [ ]:
import cv2

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
import numpy as np

In [ ]:
srs_image = cv2.imread('../images/image_1.jpg')
srs_image = cv2.cvtColor(srs_image, cv2.COLOR_BGR2RGB)

In [ ]:
srs_image.shape

In [ ]:
plt.imshow(srs_image)

In [ ]:
points = np.array([
    [500, 600],
    [1100, 600],
    [1100, 700],
    [500, 700],
])

points = np.float32(points.reshape(-1,1,2))
points.shape

In [ ]:
def drawPoints(image, points):
    plt.figure(figsize=(15,7))

    image_copy = image.copy() 
    points_to_draw = np.int32(points.reshape(-1, 2))
    
    for i in range(len(points_to_draw)-1):
        cv2.line(image_copy, tuple(points_to_draw[i]), tuple(points_to_draw[i+1]), (0, 255, 0), 2)
        cv2.circle(image_copy, tuple(points_to_draw[i]), 10, (255, 0, 0), -1)
        
    cv2.line(image_copy, tuple(points_to_draw[-1]), tuple(points_to_draw[0]), (0, 255, 0), 2)
    
    plt.imshow(image_copy)

In [ ]:
drawPoints(srs_image, points)

In [ ]:
srs_image_gray = cv2.cvtColor(srs_image, cv2.COLOR_BGR2GRAY)
srs_image_gray.shape

In [ ]:
dst_image = cv2.imread('../images/image_3.jpg')
dst_image = cv2.cvtColor(dst_image, cv2.COLOR_BGR2RGB)
dst_image_gray = cv2.cvtColor(dst_image, cv2.COLOR_BGR2GRAY)
dst_image_gray.shape

In [ ]:
plt.imshow(dst_image_gray)

In [ ]:
def drawCorners(image, corners):
    plt.figure(figsize=(15,7))

    image_copy = image.copy()

    for corner in corners:
        x, y = corner.ravel()
        cv2.circle(image_copy, (int(x), int(y)), 5, (0, 255, 0), -1)

    plt.imshow(image_copy) 

## goodFeaturesToTrack

In [ ]:
srs_corners = cv2.goodFeaturesToTrack(srs_image_gray, 100, 0.3, 10)
dst_corners = cv2.goodFeaturesToTrack(dst_image_gray, 100, 0.3, 10)

In [ ]:
drawCorners(srs_image, srs_corners)
# drawCorners(dst_image, dst_corners)

In [ ]:
srs_corners[:-1].shape, dst_corners.shape

## ORB

In [ ]:
# Initiate ORB detector
orb = cv2.ORB_create()

In [ ]:
(kpsA, descsA) = orb.detectAndCompute(srs_image_gray, None)
(kpsB, descsB) = orb.detectAndCompute(dst_image_gray, None)

In [ ]:
plt.figure(figsize=(15,7))
# draw only keypoints location,not size and orientation
img = cv2.drawKeypoints(srs_image, kpsA, None, color=(0,255,0), flags=0)
plt.imshow(img)

In [ ]:
method = cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING
matcher = cv2.DescriptorMatcher_create(method)
matches = matcher.match(descsA, descsB, None)

In [ ]:
# sort the matches by their distance (the smaller the distance,
# the "more similar" the features are)
matches = sorted(matches, key=lambda x:x.distance)

In [ ]:
len(matches)

In [ ]:
plt.figure(figsize=(15,7))

matchedVis = cv2.drawMatches(srs_image, kpsA, dst_image, kpsB, matches, None)
plt.imshow(matchedVis)

In [ ]:
# allocate memory for the keypoints (x, y)-coordinates from the
# top matches -- we'll use these coordinates to compute our
# homography matrix
ptsA = np.zeros((len(matches), 2), dtype="float")
ptsB = np.zeros((len(matches), 2), dtype="float")
# loop over the top matches
for (i, m) in enumerate(matches):
    # indicate that the two keypoints in the respective images
    # map to each other
    ptsA[i] = kpsA[m.queryIdx].pt
    ptsB[i] = kpsB[m.trainIdx].pt

In [ ]:
ptsA = ptsA[:30]
ptsB = ptsB[:30]

In [ ]:
len(ptsA), len(ptsB)

## Homography

In [ ]:
# H, mask = cv2.findHomography(srs_corners[:-1], dst_corners, cv2.RANSAC)
H, mask = cv2.findHomography(ptsA, ptsB, method=cv2.RANSAC)

In [ ]:
H

In [ ]:
mask.shape

In [ ]:
dst_points = cv2.perspectiveTransform(points, H)

In [ ]:
dst_points.shape

In [ ]:
drawPoints(dst_image, dst_points)